In [1]:
import torch
import torch.nn as nn
from torch.nn import Embedding

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import dataset
import test_lstm

2023-03-10 23:10:41.749085: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries
MU3D_path = './processed/' # 300 entries

In [3]:
# no split by person
numOfFrames = 10
X, Y = dataset.preprocessing(truth_trial_path, lie_trial_path, numOfFrames=numOfFrames)

TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
embedding_layer = Embedding(num_embeddings=numOfFrames, embedding_dim=4)

positions = torch.tensor([0,1,2,3,4,5,6,7,8,9])
embedded_positions = embedding_layer(positions)
embedded_positions

tensor([[-0.6478, -0.2735,  0.1988, -0.1061],
        [-0.2156, -0.6628,  0.0118, -0.3504],
        [-0.9394, -2.5968, -0.5656, -0.3089],
        [ 0.9634, -0.0049,  1.6088,  0.8932],
        [ 0.1580, -1.7368,  0.8074,  2.5677],
        [-0.4145, -0.9888,  1.3875, -1.8812],
        [ 0.8478,  0.5766, -1.6423,  0.2983],
        [ 0.5067, -0.4327,  0.2248,  0.0496],
        [-0.4765, -2.0241, -0.2576,  0.8703],
        [-0.6168, -0.8509, -0.7345,  0.2618]], grad_fn=<EmbeddingBackward0>)

In [6]:
#embedding
def gen_loc_list(max_frames_count, pos_encoder_size, frames_count = numOfFrames):
  embedding = nn.Embedding(max_frames_count, pos_encoder_size)
  input = torch.tensor([i for i in range(frames_count)]).clone().detach()
  loc_list = embedding(input)

  return loc_list

def gen_data(data_arr, embedding_arr):

  data_arr = torch.tensor(data_arr).clone().detach()
  list = torch.cat((data_arr, embedding_arr), 1)

  return list

In [7]:
#model

from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder

class TransformerModel(nn.Module):

  def __init__(self, inFeatCount, num_encoder_layers, n_heads = 5, n_hidden = 500, dropout = 0.3, outFeatCount = 2): #Out = [prob for 0, prob for 1]
    super(TransformerModel, self).__init__()

    encoder_layer = TransformerEncoderLayer(inFeatCount, n_heads, n_hidden, dropout)

    self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers)

    self.decoder = nn.Linear(inFeatCount, outFeatCount)

  def init_weights(self):
      initrange = 0.1
      self.encoder.weight.data.uniform_(-initrange, initrange)
      self.decoder.bias.data.zero_()
      self.decoder.weight.data.uniform_(-initrange, initrange)

  def forward(self, data):
      encoded = self.encoder(data)
      output = self.decoder(encoded)
      return torch.nn.functional.softmax(output)
  
def predict(model, inputArr):
    inputArr = inputArr.to(device).float()
    
    output = model(inputArr)
    
    return output


In [8]:
pos_encoder_size = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
#generate embedding
embedding_arr = gen_loc_list(numOfFrames, pos_encoder_size)

#generate embedded data
x_Train = []
x_Test = []

y_Train = []
y_Test = []

for i in range(xTrain.shape[0]):
    x_Train.append(gen_data(xTrain[i], embedding_arr).detach().numpy())

for i in range(xTest.shape[0]):
    x_Test.append(gen_data(xTest[i], embedding_arr).detach().numpy())

for i in range(yTrain.shape[0]):
    if yTrain[i] == 0:
        y_Train.append([[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0]])
    else:
        y_Train.append([[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]])

for i in range(yTest.shape[0]):
    if yTest[i] == 0:
        y_Test.append([[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0]])
    else:
        y_Test.append([[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]])

x_Train = torch.tensor(x_Train).to(device).float()
x_Test = torch.tensor(x_Test).to(device).float()




In [77]:
#generate model
model = TransformerModel(15, 5).to(device)

pred = model((x_Train[0]))

# add every frame of pred to get the final prediction
print(pred)

tensor([[0.3211, 0.6789],
        [0.3902, 0.6098],
        [0.4241, 0.5759],
        [0.3421, 0.6579],
        [0.3605, 0.6395],
        [0.5169, 0.4831],
        [0.4748, 0.5252],
        [0.3582, 0.6418],
        [0.3499, 0.6501],
        [0.4221, 0.5779]], grad_fn=<SoftmaxBackward0>)


/var/folders/66/pn8653k51_75x5ksbpsy8hm80000gn/T/ipykernel_24652/476567611.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.nn.functional.softmax(output)


In [21]:
# training
import torch.optim as optim

#generate model
model = TransformerModel(15, 5).to(device)

def train(model, x_train, y_train, x_test, y_test, epochs = 100, lr = 0.001, batch_size = 10):
    """ Train a model on a dataset """

    loss_items = []
    accuracy_items = []
    test_accuracy_items = []
    
    # create a data loader to handle batching
    xTrain_loader =  torch.utils.data.DataLoader(x_train, batch_size=batch_size, shuffle=False)
    xTest_loader = torch.utils.data.DataLoader(x_test, batch_size=batch_size, shuffle=False)

    # create a loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # train the model
    for epoch in range(epochs):

        # train

        idx = 0
        model.train()
        for batch in xTrain_loader:

            # get data
            x_train_data = batch.to(device).float()
            y_train_data = torch.tensor(y_train[idx]).to(device)

            # forward pass
            y_pred = model(x_train_data)

            # compute loss
            loss = loss_fn(y_pred, y_train_data)

            # backward pass
            loss.backward()

            # update weights
            optimizer.step()
            optimizer.zero_grad()

            if idx == 0:
                print(y_pred.sum(0).detach().numpy())
                print(y_train_data)

        # evaluate
        model.eval()

        test_acc = 0
        acc = 0
        tot_loss = 0

        with torch.no_grad():
            
            idx_test = 0
            for batch in xTest_loader:

                x_test_data = batch.to(device).float()
                y_test_data = torch.tensor(y_test[idx_test]).to(device)

                y_pred = model(x_test_data)

                # compute test accuracy
                test_acc += (y_pred == y_test_data).type(torch.float).mean().item()
                idx_test += 1
                

            test_acc /= len(xTest_loader)  
            test_accuracy_items.append(test_acc)

            idx_train = 0
            for batch in xTrain_loader:
                x_train_data = batch.to(device).float()
                y_train_data = torch.tensor(y_train[idx_train]).to(device)

                y_pred = model(x_train_data)

                # compute accuracy
                acc += (y_pred == y_train_data).type(torch.float).mean().item()
                idx_train += 1

                tot_loss += loss_fn(y_pred, y_train_data).item()

            acc /= len(xTrain_loader)
            accuracy_items.append(acc)

        # store loss and accuracy
        loss_items.append(tot_loss)

        # print progress
        if (epoch+1) % 1 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {tot_loss:.4f}, Accuracy: {acc:.4f}, Test Accuracy: {test_acc:.4f}')

        idx += 1
            

train(model, x_Train, y_Train, x_Test, y_Test)

/var/folders/66/pn8653k51_75x5ksbpsy8hm80000gn/T/ipykernel_24652/476567611.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.nn.functional.softmax(output)


[[1.         0.99999994]
 [1.         1.        ]
 [1.         1.        ]
 [1.         1.0000001 ]
 [1.0000001  1.        ]
 [1.0000001  1.        ]
 [0.99999994 1.        ]
 [0.99999994 1.        ]
 [0.9999999  1.        ]
 [1.         0.9999999 ]]
tensor([[1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0]])
[[1.        1.       ]
 [1.        1.       ]
 [1.        1.       ]
 [1.0000001 1.       ]
 [1.        1.       ]
 [1.        1.       ]
 [1.        1.       ]
 [1.        1.0000001]
 [1.        1.       ]
 [1.0000001 1.       ]]
tensor([[1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0]])
[[1.         1.        ]
 [1.         1.        ]
 [1.0000001  1.0000001 ]
 [1.         1.        ]
 [1.         1.        ]
 [1.         1.0000001 ]
 [1.         1.        ]
 [0.99999994 0.999999

KeyboardInterrupt: 